**Week 10: Markovian Langevin Dynamics and Diffusion**

Jannek Schaffert, Frédéric Sonak

This markdown was created with the assistance of ChatGPT, focusing on grammar, spelling, and readability.

**Background**

In this excercise a Langevian Integrator (LE) with Markovian dynamics is implemented.



**Task 1: Implementation**

equation:
$
\vec{v}_{k+1} = \vec{v}_k - \frac{1}{m} \left( \Gamma \vec{v}_k \Delta t - \sqrt{2 \Gamma k_B T \Delta t} \cdot \vec{R}_k \right);
$

with

$
\vec{x}_{k+1} = \vec{x}_k + \vec{v}_k \Delta t
$.

With $k$ denoting the step number, $\Delta t$ the time step length, $U$ the potential energy, $x$ the position, $v$ the velocity, and a friction constant $\Gamma$.




Importing all libraries used in this code.


In [1]:
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

from numba import njit

import copy
import numpy as np
import scipy as scy

**Task 2 - LE Simulation**

A particle with a mass of $m = 18$g/ mol and a starting velocity corresponding to a temperature of $T = 293.15$K in a random direction is assigned to the particle.
The potential energy is set a constant throughout the box. This changes our LE integrator. TO observe this, first we need to rewrite the Langevian equation with potential energy $U$:

$
\vec{v}_{k+1} = \vec{v}_k - \frac{1}{m} \left( \frac{dU}{dx} \Delta t + \vec{v}_k \Gamma \Delta t - \sqrt{2 \Gamma k_B T \Delta t} \cdot \vec{R}_k \right)
$

Since the Potential is set to a constant the equation simplifies to

$
\vec{v}_{k+1} = \vec{v}_k - \frac{1}{m} \left( \vec{v}_k \Gamma \Delta t - \sqrt{2 \Gamma k_B T \Delta t} \cdot \vec{R}_k \right)
$

and

$
\vec{x}_{k+1} = \vec{x}_k + \vec{v}_k \Delta t
$

.


In [1]:
def langevian_integrator(steps, x0, v0, mass, gamma, temp, k_B = 1.380649e-23, verbose = False):

    array = np.random.normal(loc=0.0, scale=1.0, size = (steps, 2))

    #initialise position and velocity arrays
    positions = np.zeros((steps, len(x0)))  # Position array
    velocities = np.zeros((steps, len(v0)))  # Velocity array

    # add the zero poisitoin and velocuty
    x[0] = x0
    v[0] = v0

    #loop over steps
    for i in range(steps):

        R = rng.normal(0, 1, size=x0.shape)  # Gaussian random noise
        v = velocities[-1]
        x = positions[-1]

        # Update velocity and position
        v_next = v - (gamma / mass) * v * dt + np.sqrt(2 * gamma * k_B * temp * dt) / mass * R
        x_next = x + v_next * dt

        velocities.append(v_next)
        positions.append(x_next)

        return np.array(positions), np.array(velocities)

In [2]:
# Setting all parameters

steps = 1000000
gamma = 50
dt = 1.0e-6 #ns
mass = 0.018 # kg/mol
R = 1.380649e-23  # Boltzmann constant (J/K)
temp = 293.15

#start position and velocity
x_0 = np.array([0.0, 0.0])
v_0 = np.array([0.0, 0.0])



NameError: name 'np' is not defined